In [1]:
import keras
from keras.layers import LSTM, Dropout, Dense
import tensorflow as tf
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [3]:
data = pd.read_csv("/data/creditcard-fraud.csv.gz")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

In [5]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
rec_count = len(data)
rec_count

284807

In [7]:
sequence_length = 10

In [8]:
def generate_data(X, y, sequence_length = 10, step = 1):
    X_local = []
    y_local = []
    for start in range(0, len(data) - sequence_length, step):
        end = start + sequence_length
        X_local.append(X[start:end])
        y_local.append(y[end-1])
    return np.array(X_local), np.array(y_local)

X_sequence, y = generate_data(data.loc[:, "V1":"V28"].values, data.Class)

In [9]:
X_sequence.shape, y.shape

((284797, 10, 28), (284797,))

In [14]:
model = keras.Sequential()
model.add(LSTM(100, input_shape = (10, 28)))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy"
              , metrics=[keras.metrics.binary_accuracy]
              , optimizer="adam")

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               51600     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 51,701
Trainable params: 51,701
Non-trainable params: 0
_________________________________________________________________


In [15]:
training_size = int(len(X_sequence) * 0.7)

In [16]:
X_train, y_train = X_sequence[:training_size], y[:training_size]
X_test, y_test = X_sequence[training_size:], y[training_size:]

In [18]:
model.fit(X_train, y_train, batch_size=64, epochs=50)

Epoch 1/50
199357/199357 [==============================] - 30s 149us/step - loss: 0.0037 - binary_accuracy: 0.9993
Epoch 2/50
199357/199357 [==============================] - 32s 161us/step - loss: 0.0036 - binary_accuracy: 0.9993
Epoch 3/50
199357/199357 [==============================] - 31s 155us/step - loss: 0.0034 - binary_accuracy: 0.9993
Epoch 4/50
199357/199357 [==============================] - 31s 156us/step - loss: 0.0034 - binary_accuracy: 0.9993
Epoch 5/50
199357/199357 [==============================] - 33s 166us/step - loss: 0.0032 - binary_accuracy: 0.9994
Epoch 6/50
199357/199357 [==============================] - 42s 213us/step - loss: 0.0030 - binary_accuracy: 0.9993
Epoch 7/50
199357/199357 [==============================] - 32s 162us/step - loss: 0.0029 - binary_accuracy: 0.9993
Epoch 8/50
199357/199357 [==============================] - 38s 188us/step - loss: 0.0026 - binary_accuracy: 0.9994
Epoch 9/50
199357/199357 [==============================] - 37s 186us/st

In [19]:
model.evaluate(X_test, y_test)

85440/85440 [==============================] - 4s 50us/step


[0.005906340044841124, 0.9995435393258427]

In [20]:
y_test_prob = model.predict(X_test, verbose=1)

85440/85440 [==============================] - 4s 48us/step


In [21]:
y_test_pred = np.where(y_test_prob > 0.5, 1, 0)

In [22]:
from sklearn.metrics import confusion_matrix

In [23]:
confusion_matrix(y_test, y_test_pred)

array([[85328,     4],
       [   35,    73]])